In [ ]:
pip install Kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download valentynsichkar/Traffic-Signs-Preprocessed

In [2]:
!mkdir Traffic-Signs-Preprocessed

In [ ]:
! unzip /content/Traffic-Signs-Preprocessed.zip -d Traffic-Signs-Preprocessed

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def load_data(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data
data0_path = '/content/Traffic-Signs-Preprocessed/data0.pickle'
data8_path = '/content/Traffic-Signs-Preprocessed/data8.pickle'

data0 = load_data(data0_path)
data8 = load_data(data8_path)

def reshape_data(data, target_shape):
    if data.shape[1:] != target_shape:
        data = data.transpose(0, 2, 3, 1)
    return data

data0['x_train'] = reshape_data(data0['x_train'], (32, 32, 3))
data0['x_validation'] = reshape_data(data0['x_validation'], (32, 32, 3))
data0['x_test'] = reshape_data(data0['x_test'], (32, 32, 3))

data8['x_train'] = reshape_data(data8['x_train'], (32, 32, 1))
data8['x_validation'] = reshape_data(data8['x_validation'], (32, 32, 1))
data8['x_test'] = reshape_data(data8['x_test'], (32, 32, 1))


def preprocess_data(data):
    data['x_train'] = data['x_train'] / 255.0
    data['x_validation'] = data['x_validation'] / 255.0
    data['x_test'] = data['x_test'] / 255.0

    num_classes = len(np.unique(data['y_train']))
    data['y_train'] = to_categorical(data['y_train'], num_classes)
    data['y_validation'] = to_categorical(data['y_validation'], num_classes)
    data['y_test'] = to_categorical(data['y_test'], num_classes)

    return data

data0 = preprocess_data(data0)
data8 = preprocess_data(data8)

def build_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

input_shape_data0 = (32, 32, 3)
input_shape_data8 = (32, 32, 1)
num_classes = data0['y_train'].shape[1]

model_data0 = build_model(input_shape_data0, num_classes)
model_data8 = build_model(input_shape_data8, num_classes)

model_data0.summary()
model_data8.summary()

history_data0 = model_data0.fit(data0['x_train'], data0['y_train'],
                                epochs=10,
                                batch_size=64,
                                validation_data=(data0['x_validation'], data0['y_validation']))

history_data8 = model_data8.fit(data8['x_train'], data8['y_train'],
                                epochs=10,
                                batch_size=64,
                                validation_data=(data8['x_validation'], data8['y_validation']))


In [6]:
def load_and_visualize_sample(data, dataset_name='data'):
    random_idx = np.random.randint(data['x_train'].shape[0])
    sample_image = data['x_train'][random_idx]
    sample_label = np.argmax(data['y_train'][random_idx])

    plt.imshow(sample_image.squeeze(), cmap='gray' if sample_image.shape[-1] == 1 else None)
    plt.title(f'Sample from {dataset_name} - Label: {sample_label}')
    plt.axis('off')
    plt.show()


In [ ]:
# Evaluate models
def evaluate_model(model, x_test, y_test):
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_true, y_pred_classes)
    f1 = f1_score(y_true, y_pred_classes, average='weighted')
    recall = recall_score(y_true, y_pred_classes, average='weighted')
    precision = precision_score(y_true, y_pred_classes, average='weighted')

    return accuracy, f1, recall, precision

# Evaluate model_data0
accuracy_data0, f1_data0, recall_data0, precision_data0 = evaluate_model(model_data0, data0['x_test'], data0['y_test'])

# Evaluate model_data8
accuracy_data8, f1_data8, recall_data8, precision_data8 = evaluate_model(model_data8, data8['x_test'], data8['y_test'])

# Print evaluation results
print("Evaluation results for model_data0:")
print(f"Accuracy: {accuracy_data0:.4f}")
print(f"F1 Score: {f1_data0:.4f}")
print(f"Recall: {recall_data0:.4f}")
print(f"Precision: {precision_data0:.4f}")
print()

print("Evaluation results for model_data8:")
print(f"Accuracy: {accuracy_data8:.4f}")
print(f"F1 Score: {f1_data8:.4f}")
print(f"Recall: {recall_data8:.4f}")
print(f"Precision: {precision_data8:.4f}")